Extract the evidence in each instance provided by SHAP

In [1]:
import json
import numpy as np

with open("SHAP_values/combined_shap_values.json", "r") as f:
    shap_data = json.load(f)

with open("../data/annotated_labels.json", "r") as f:
    annotated_labels = json.load(f)

with open("../data/classified_labels.json", "r") as f:
    classified_labels = json.load(f)

with open("../data/classified_labels_three_labels.json", "r") as f:
    classified_labels_three_labels = json.load(f)

with open('../data/original_post_tokens.json', 'r', encoding='utf-8') as f:
    original_post_tokens = json.load(f)

In [2]:
# shap_values = np.array(shap_data["values"][15])
# print(shap_values[:, 1])
# print(np.array(shap_data["values"][15]))

In [3]:
def map_shap_words_to_original(original_post_token,
                               shap_words):  # map sub-words generated by SHAP to its possible original word
    mapped_words = []

    for shap_word in shap_words:
        shap_word = shap_word.strip().lower()

        # if len(shap_word) <= 1 or shap_word.isspace():
        #     continue
        if shap_word.isspace():
            continue

        best_match = None
        min_distance = float('inf')

        for word in original_post_token:
            word_lower = word.lower()
            if shap_word in word_lower:
                start = word_lower.find(shap_word)
                end = start + len(shap_word)
                distance = min(abs(start), abs(len(word_lower) - end))
                if distance < min_distance:
                    min_distance = distance
                    best_match = word

        if best_match:
            mapped_words.append(best_match)

    return list(set(mapped_words))

# def select_important_words_by_cumulative_contribution(shap_values, sentence_length, contribution_threshold=0.5):
#     N = 5
#     max_words = max(1, (sentence_length + N - 1) // N)  # 例如，当 N=5 时，长度5-9的句子选择2个单词，10-14选择3个
# 
#     total_contribution = np.abs(shap_values).sum()
#     # total_contribution = shap_values.sum()
#     # print(total_contribution)
#     sorted_indices = np.argsort(np.abs(shap_values))
#     # print(sorted_indices)
#     cumulative_contribution = 0.0
#     selected_indices = []
# 
#     for idx in sorted_indices:
#         cumulative_contribution += np.abs(-shap_values[idx])
#         selected_indices.append(idx)
#         # if total_contribution == 0:
#         #     break
#         if cumulative_contribution / total_contribution >= contribution_threshold or len(selected_indices) >= max_words:
#             break
# 
#     return selected_indices

In [4]:
# important_words = []
# 
# for idx, label in enumerate(classified_labels):
#     if label == 1:
#         shap_values = np.array(shap_data["values"][idx])
#         words = shap_data["data"][idx]
#         
#         sentence_length = len(words)
#         # if idx == 131:
#         #     print(sentence_length)
# 
#         selected_indices_hatespeech = select_important_words_by_cumulative_contribution(shap_values[:, 0], sentence_length)
#         selected_indices_offensive = select_important_words_by_cumulative_contribution(shap_values[:, 2], sentence_length)
# 
#         top_indices = list(set(selected_indices_hatespeech) | set(selected_indices_offensive))
# 
#         important_words_for_entry = [words[i] for i in top_indices]
#         mapped_words = map_shap_words_to_original(original_post_tokens[idx], important_words_for_entry)
# 
#         # important_words.append({
#         #     "entry_index": idx,
#         #     "important_words": mapped_words,
#         #     "important_words_for_entry": important_words_for_entry
#         # })
#         important_words.append(mapped_words)
#     else:
#         important_words.append([])

# for instance in important_words:
#     print(instance["entry_index"], instance["important_words"])
#     if instance["entry_index"] == 131:
#         print(instance["important_words_for_entry"])

# save the important words into files
# with open("../data/important_words_abusive_SHAP.json", "w") as f:
#     json.dump(important_words, f, indent=4)

Because the previous code cell does not generate the important words for normal speech, now I have modified it to include all the instances.

In [5]:
# def find_top_values(values_list):
#     positive_indices = np.where(values_list > 0)[0]
#     positive_values = values_list[positive_indices]
#     top_5_indices_in_positive = np.argsort(positive_values)[-5:][::-1]
#     top_5_original_indices = positive_indices[top_5_indices_in_positive]
#     return top_5_original_indices
# print(find_top_values(np.array([0.1, -0.3, 0.5, 0.2, 0, -0.1, 1.2, 0.4, 0.9, -0.5])))

In [6]:
# import numpy as np

def find_top_values(values_list):  #top 5 values' indexes
    positive_indices = np.where(values_list > 0)[0]
    positive_values = values_list[positive_indices]
    top_5_indices_in_positive = np.argsort(positive_values)[-5:][::-1]
    top_5_original_indices = positive_indices[top_5_indices_in_positive]
    return top_5_original_indices


def extract_important_words(shap_data_original, labels_three_classes):
    important_words_all = []  # include all 3 classes
    for index, labeled_class_int in enumerate(labels_three_classes):
        extracted_shap_values = np.array(shap_data_original["values"][index])
        shap_words = shap_data_original["data"][index]  # words or sub-words list, then we need to map it back to the original words

        shap_values_of_one_class = extracted_shap_values[:, labeled_class_int]  # choose the right label column
        selected_indices_for_the_class = find_top_values(shap_values_of_one_class)
        important_words_for_single_entry = [shap_words[ind] for ind in selected_indices_for_the_class]
        possible_mapped_words = map_shap_words_to_original(original_post_tokens[index],
                                                           important_words_for_single_entry)
        important_words_all.append(possible_mapped_words)
    return important_words_all

important_words = extract_important_words(shap_data, classified_labels_three_labels)

# for i, word in enumerate(important_words[1500:1700]):
#     print(i, word)
# for instance in important_words:
#     print(instance["entry_index"], instance["important_words"])
#     if instance["entry_index"] == 131:
#         print(instance["important_words_for_entry"])


with open("../data/important_words_SHAP.json", "w") as f:
    json.dump(important_words, f, indent=4)

In [7]:
import json

# with open('../data/important_words_abusive_SHAP.json', 'r') as f:
#     important_words_data = json.load(f)
    
with open('../data/important_words_SHAP.json', 'r') as f:
    important_words_data = json.load(f)

with open('../data/rationale_tokens_list.json', 'r') as f:
    rationale_tokens_data = json.load(f)


spanr_count = 0
spanr_total = 0
cover_total = 0
cover_correct = 0

# rationale_tokens_data = rationale_tokens_data_full[:11500]

for i in range(len(annotated_labels)):
    if annotated_labels[i] == 1:
        rationale_tokens = set(rationale_tokens_data[i])
        # important_tokens = set(important_words_data[i]["important_words"])
        important_tokens = set(important_words_data[i])

        spanr_total += 1  
        if rationale_tokens & important_tokens:
            # print(i, rationale_tokens & important_tokens)
            spanr_count += 1
            cover_correct += len(rationale_tokens & important_tokens)
        cover_total += len(rationale_tokens)

if cover_total > 0:
    cover = cover_correct / cover_total
else:
    cover = 0

spanr = spanr_count / spanr_total

print(f"SpanR: {spanr}")
print(f"Cover: {cover}")
print(spanr_total, spanr_count)
print(cover_correct, cover_total)


SpanR: 0.8857629317334198
Cover: 0.2702140557715624
12334 10925
25853 95676
